In [30]:
import numpy as np
import cv2
import tensorflow as tf
import sklearn as sk
import sklearn.model_selection
from imutils import paths
from skimage import io

tf.__version__

'2.1.0'

In [16]:
#INIT_LR = 1e-3
#EPOCHS = 10
#BS = 8
dataset = "C:/Users/User/Documents/TESTE_NODE/Robo-Imagens-Gatos/content_pixabay"
dataset

'C:/Users/User/Documents/TESTE_NODE/Robo-Imagens-Gatos/content_pixabay'

In [17]:
args={}
args["dataset"]=dataset
args

{'dataset': 'C:/Users/User/Documents/TESTE_NODE/Robo-Imagens-Gatos/content_pixabay'}

In [42]:
iPaths = list(paths.list_images(args["dataset"]))  #image paths
data = []
labels = []
for iPath in iPaths:
    label = iPath.split(os.path.sep)[-2]   #split the image paths
    image = io.imread(iPath)#cv2.imread(iPath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Convert images into RGB Channel
    image = cv2.resize(image, (224, 224))  #Resizing the images
    data.append(image)
    labels.append(label)
data = np.array(data) / 255.0
labels = np.array(labels)
labels

array(['Bicolor', 'Bicolor', 'Bicolor', 'Calico', 'Calico', 'Calico',
       'Calico', 'Calico', 'Calico', 'Calico', 'Calico', 'Calico',
       'Calico', 'Calico', 'Calico', 'Calico', 'Calico', 'Calico',
       'Calico', 'Calico', 'Calico', 'Calico', 'Calico', 'Calico',
       'Calico', 'Calico', 'Calico', 'Calico', 'Calico', 'Calico',
       'Calico', 'Calico', 'Calico', 'Calico', 'Calico', 'Calico',
       'Calico', 'Calico', 'Hairless', 'Hairless', 'Hairless', 'Hairless',
       'Hairless', 'Hairless', 'Hairless', 'Hairless', 'Hairless',
       'Hairless', 'solid_color_black', 'solid_color_black',
       'solid_color_black', 'solid_color_black', 'solid_color_black',
       'solid_color_black', 'solid_color_black', 'solid_color_black',
       'solid_color_black', 'solid_color_black', 'solid_color_black',
       'solid_color_black', 'solid_color_black', 'solid_color_black',
       'solid_color_black', 'solid_color_black', 'solid_color_black',
       'solid_color_black', 'solid_color_b

In [19]:
data

array([[[[0.29803922, 0.51764706, 0.64313725],
         [0.23137255, 0.5254902 , 0.59215686],
         [0.31764706, 0.63529412, 0.64313725],
         ...,
         [0.48627451, 0.72941176, 0.77254902],
         [0.54117647, 0.78039216, 0.81176471],
         [0.54901961, 0.8       , 0.81176471]],

        [[0.37647059, 0.58823529, 0.71764706],
         [0.23137255, 0.52941176, 0.6       ],
         [0.31764706, 0.61568627, 0.65098039],
         ...,
         [0.45490196, 0.72156863, 0.74901961],
         [0.41568627, 0.69019608, 0.69803922],
         [0.48627451, 0.76470588, 0.75294118]],

        [[0.38823529, 0.61960784, 0.72156863],
         [0.22745098, 0.56078431, 0.6       ],
         [0.41176471, 0.71372549, 0.75686275],
         ...,
         [0.41176471, 0.69019608, 0.71764706],
         [0.33333333, 0.63921569, 0.63529412],
         [0.41568627, 0.70980392, 0.68235294]],

        ...,

        [[0.16078431, 0.34509804, 0.28627451],
         [0.14509804, 0.30588235, 0.28627451]

In [43]:
LB = sk.preprocessing.MultiLabelBinarizer()  #Initialize label binarizer
labelsCat = LB.fit_transform(labels)
labelsCat = tf.keras.utils.to_categorical(labelsCat); print(labelsCat)

[[[0. 1.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]
  [1. 0.]
  ...
  [1. 0.]
  [1. 0.]
  [1. 0.]]

 ...

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [0. 1.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  ...
  [0. 1.]
  [1. 0.]
  [1. 0.]]]


In [44]:
(X_train, X_test, Y_train, Y_test) = sk.model_selection.train_test_split(data, labelsCat,
    test_size=0.33, stratify=labelsCat, random_state=42)
X_train

ValueError: Found array with dim 3. Estimator expected <= 2.